In [17]:
import pandas as pd

df = pd.read_csv(
    "dataset.csv",
    encoding="latin-1",
    engine="python",
    on_bad_lines="skip"
)

df.head()


,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [18]:
df = df[[
    "votes",
    "approx_cost(for two people)",
    "online_order",
    "book_table",
    "location",
    "cuisines"
]]


In [19]:
df.dropna(inplace=True)


In [20]:
df["approx_cost(for two people)"] = df["approx_cost(for two people)"].str.replace(",", "")
df["approx_cost(for two people)"] = df["approx_cost(for two people)"].astype(float)


In [21]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df["location"] = le.fit_transform(df["location"])
df["online_order"] = le.fit_transform(df["online_order"])
df["book_table"] = le.fit_transform(df["book_table"])
df["cuisines"] = le.fit_transform(df["cuisines"])


In [27]:
top_cuisines = df["cuisines"].value_counts().head(12).index
df = df[df["cuisines"].isin(top_cuisines)]


In [28]:
X = df.drop("cuisines", axis=1)
y = df["cuisines"]


In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [30]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=300, solver="lbfgs")
model.fit(X_train, y_train)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=300)

In [31]:
y_pred = model.predict(X_test)


In [32]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, y_pred)
accuracy


0.27030625832223704

In [33]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

         232       0.00      0.00      0.00        90
         238       0.00      0.00      0.00        94
         347       0.00      0.00      0.00        92
         503       0.00      0.00      0.00        81
         717       0.00      0.00      0.00        64
        1038       0.00      0.00      0.00        76
        1093       0.00      0.00      0.00        89
        1290       0.00      0.00      0.00        38
        1573       0.24      0.58      0.34       297
        1639       0.22      0.34      0.27       245
        2110       0.37      0.65      0.47       234
        2190       0.00      0.00      0.00       102

    accuracy                           0.27      1502
   macro avg       0.07      0.13      0.09      1502
weighted avg       0.14      0.27      0.18      1502



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Model Performance Analysis and Bias Discussion

The cuisine classification model was trained using Logistic Regression to predict restaurant cuisine types based on features such as votes, approximate cost for two people, online ordering availability, table booking, and location.

The model achieved an accuracy of approximately **27%**, which is acceptable for multi-class cuisine classification due to the large number of cuisine categories and overlapping restaurant characteristics.

It was observed that the model often confuses cuisines such as **North Indian** and **South Indian** because restaurants belonging to these cuisines share similar pricing ranges, locations, and ordering patterns. This similarity in features makes it difficult for the model to clearly distinguish between these cuisines.

In contrast, cuisines such as **Italian** and **Continental** are predicted more accurately as they usually have distinct pricing and restaurant features.

This analysis highlights the challenge of cuisine classification using limited numerical features and demonstrates how data similarity and class imbalance can introduce bias in machine learning models.
